## qiskit_circ_make

### **Outline**
This function automatically constructs a quantum circuit that performs Block-Encoding using `Qiskit`, based on the provided input.

### **Index List**

|argument name|type|role|
|---|---|---|
|gate_inf|str|Information about the production of Pauli matrices|
|zero_one|list(elements:int)|Information about the state of control qubits|
|circ|QuantumCircuit(qulacs)|Gate-targeting quantum circuit|
|qubit|int|the number of qubits|
|ancilla|int|the number of ancilla qubits|

### **Python code**
```python
def qiskit_circ_make(gate_inf, zero_one, circ, qubit, ancilla):
    """
    This function automatically constructs a quantum circuit for Qiskit that performs block encoding based on the given information.

    Parameters:
        gate_inf: the information of Pauli matrix product
        zero_one: the information of control qubits
        circ: a pre-defined quantum circuit used for actual computation
        qubit: the number of qubits which express the quantum state
        ancilla: the number of ancilla qubits for Block-Encoding 

    Returns:
        None. 
        A quantum circuit implementing Pauli operators with control qubits is created on circ.
    """
    # Local Values
    work_ope_order = [] # A list to store information about Pauli matrix product
    input_switch = 0 # A switch to initiate reading information about Pauli matrix product
    input_ele = "" # A temporary list to store information about Pauli matrix product
    # If Pauli matrix product is Identity,
    if gate_inf == "":
        gate_a = [[1., 0.],
                  [0., 1.]]
        unitary_gate = Operator(np.array(gate_a)).to_instruction()
        for i in range(int(qubit - ancilla)):
            circ.append(unitary_gate, [i])
    # If Pauli matrix product is the product of some Pauli matirces (ex: X0Y2Z4Y5)
    else:
        # Read the Pauli operators that make up the Pauli matrix product
        for i in range(len(gate_inf)):
            # Determine whether to interpret each item as a coefficient, a Pauli matrix product, or to ignore it.
            # 0: Read, 1: ignore
            if i < len(gate_inf) - 1.5:
                if input_switch > 0.3:
                    work_ope_order.append(input_ele)
                    input_ele = ""
                    input_switch = 0
                if gate_inf[i+1] == "X":
                    input_switch = 1
                if gate_inf[i+1] == "Y":
                    input_switch = 1
                if gate_inf[i+1] == "Z":
                    input_switch = 1
                if gate_inf[i+1] == "I":
                    input_switch = 1
                input_ele += gate_inf[i]
            else:
                work_ope_order.append(input_ele)
                input_switch = 0
                work_ope_order[-1] += gate_inf[-1]
        # Construct a quantum circuit from the reading results
        for i in range(len(work_ope_order)):
            num_inf = ""
            for j in range(len(work_ope_order[i])-1):
                num_inf += work_ope_order[i][j+1]
            tag_num = int(num_inf)
            gate_pos = qubit - tag_num - 1
            gate_pos = ancilla + tag_num
            cont_list = []
            for j in range(ancilla):
                cont_list.append(j)
            cont_list.append(gate_pos)
            if work_ope_order[i][0] == "X":
                add_gate = XGate().control(ancilla, ctrl_state=zero_one)
                circ.append(add_gate, cont_list)
            elif work_ope_order[i][0] == "Y":
                add_gate = YGate().control(ancilla, ctrl_state=zero_one)
                circ.append(add_gate, cont_list)
            elif work_ope_order[i][0] == "Z":
                add_gate = ZGate().control(ancilla, ctrl_state=zero_one)
                circ.append(add_gate, cont_list)
            elif work_ope_order[i][0] == "I":
                add_gate = IGate().control(ancilla, ctrl_state=zero_one)
                circ.append(add_gate, cont_list)
```

### **Sample Run**

In [ ]:
import numpy as np
import pitbe
import qiskit
#from qiskit import QuantumCircuit
from qiskit.circuit.library import ZGate, XGate, YGate, IGate

In [ ]:
circuit = qiskit.QuantumCircuit(4)
ope_lst = ["I0I1", "X0X1", "Y0Y1", "Z0I1"]
cont_list = ["00", "01", "10", "11"]

for j in range(len(cont_list)):
    pitbe.qiskit_circ_make(ope_lst[j], cont_list[j], circuit, 4, 2)

### **Constructed Quantum Circuit**

![Quantum Circuit](../../picture/pennydemo2.jpeg)

### **Caution**

This function is intended for use with `Qiskit` and may not execute correctly with other platforms.\
For `Qulacs`, please use the dedicated function provided for that framework.